In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from load_files import load_data
import logging
from tqdm import tqdm
import time

In [2]:
logfilename = "test_log.log"
logger = logging.getLogger(__name__)
logging.basicConfig(filename=logfilename, encoding='utf-8', level=logging.DEBUG)
logger.info("Starting the program")


In [3]:
# loading the entire folder crashes the kernel. Out of memory.
data = load_data("/home/clyholm/ml-lora-communication/plot_stuff/csv", logger=logger)

: 

In [ ]:
# test print to check that symbol increases
print(data['symbol'][0])

In [6]:
def generate_plots(data, logger, spreading_factor: int, num_samples: int, directory: str):
    
    sample_idx = 0
    start_time = time.time()
    num_symbols = 2**spreading_factor
    plt.switch_backend('agg')
    
    for i in tqdm(range(len(data)), desc="Generating plots"):
        freqs_idx = np.arange(0, len(data['freqs'][i]), 1)
        
        fig = plt.figure(figsize=(1,1), dpi=num_symbols)
        ax = fig.add_subplot(111)
        ax.set_ylim(0, 130)
        plt.axis('off')
        fig.set_facecolor('black')
        plt.plot(freqs_idx, data['freqs'][i], color = 'white', linewidth=0.5)
        plt.close(fig)
        
        sample_idx += 1
        if sample_idx > num_samples:
            sample_idx = 1
        
        plots_dir = os.path.join(directory, "plots")
        os.makedirs(plots_dir, exist_ok=True)
        #save images to folder
        try:
            fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
            fig.savefig(os.path.join(plots_dir, f"snr_{data['snr'][i]}_symbol_{data['symbol'][i]}_{sample_idx}.png"), dpi=num_symbols)
        except Exception as e:
            logger.error(f"Error generating plot for sample {sample_idx} in file snr_{data['snr'][i]}_symbol_{data['symbol'][i]}. Error: {e}")
        
        if (i + 1) % 5000 == 0:
            logger.info(f"Generated {i + 1} plots in {time.time() - start_time:.4f} seconds") 

    logger.info(f"Finished generating {len(data)} plots in {time.time() - start_time:.4f} seconds")
        

In [ ]:
# test the function
# THIS WILL FLOOD THE OUTPUT DIRECTORY WITH PLOTS
generate_plots(data, logger, 7, 1000, "/home/clyholm/ml-lora-communication/output")

In [ ]:
# find the limit for y axis
# Flatten the lists in the 'freqs' column while keeping track of their original row and index
flattened_freqs = [(value, row_idx, col_idx) 
                   for row_idx, sublist in enumerate(data['freqs']) 
                   for col_idx, value in enumerate(sublist)]

# Find the maximum value and its location
max_value, max_row_idx, max_col_idx = max(flattened_freqs, key=lambda x: x[0])

print(f"The absolute maximum value is: {max_value}")
print(f"Location: Row {max_row_idx}, Index {max_col_idx}")
print(f"Symbol: {data['symbol'][max_row_idx]}")
print(f"SNR: {data['snr'][max_row_idx]}")

In [ ]:
# inspect plots with y axis constraints
freqs_idx = np.arange(0, len(data['freqs'][max_row_idx]), 1)

fig = plt.figure(figsize=(1,1), dpi=128)
ax = fig.add_subplot(111)
ax.set_ylim(0, 380)
plt.plot(freqs_idx, data['freqs'][88449], linewidth=0.5)
plt.show()

# There is a big outlier in the -16 snr